Questions that interest us in our analysis
1. What features have are often null? Are there patterns that can be identified and associated with our target feature
2. What is the propensity of Sepsis within the train population?
    1. How is that affected by demographic/contextual factors? (Severe Sepsis has been found to be more prevelant in male patients. See ncbi.nlm.nih.gov/pmc/articles/PMC3916365/) We will check these factors using hypothesis testing
3. What is the correlation between feature values and sepsis?
    1. Within a patients history, what correlations can be found?
    2. Examining across all patients, the features at 6 hours before and the features at diagnosis of Sepsis
    3. What differences can be noted between members who did not get Sepsis and those that did? (Hypothesis Testing)

In [122]:
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

from scipy.stats import chi2_contingency

In [3]:
train_dir = '/home/student/Data-analysis-and-presentation/HW1/data/train'

We will begin with a basic examination of the overal structure of the data

We will note that we have 40 features and 1 label

In [65]:
# example_df = pd.read_csv(train_dir + '/patient_14421.psv',sep='|') # has sepsis
example_df = pd.read_csv(train_dir + '/patient_0.psv',sep='|')
print(example_df.shape)
example_df.head()

(23, 41)


,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,75.91,0,0,1,-98.6,1,0
1,61.0,99.0,36.44,124.0,65.0,43.0,17.5,NaN,NaN,NaN,...,NaN,NaN,NaN,75.91,0,0,1,-98.6,2,0
2,64.0,98.0,NaN,125.0,64.0,41.0,27.0,NaN,NaN,NaN,...,NaN,NaN,NaN,75.91,0,0,1,-98.6,3,0
3,56.0,100.0,NaN,123.0,65.0,41.0,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,75.91,0,0,1,-98.6,4,0
4,66.0,99.0,NaN,120.0,67.0,43.0,23.0,NaN,NaN,NaN,...,NaN,NaN,NaN,75.91,0,0,1,-98.6,5,0


In [89]:
# print(example_df.SepsisLabel.sum())
# print(example_df.shape)
# print(example_df.shape[0] - example_df.SepsisLabel.sum())
# print(example_df.loc[example_df.shape[0] - example_df.SepsisLabel.sum() - 1 , 'SepsisLabel'])

# first_row = example_df.shape[0] - example_df.SepsisLabel.sum() 

# example_df.iloc[:first_row].count()/ example_df.iloc[:first_row].shape[0]
example_df.mean()[:-7]



HR                   60.954545
O2Sat                97.000000
Temp                 36.165000
SBP                 136.600000
MAP                  66.704545
DBP                  44.066667
Resp                 14.236842
EtCO2                      NaN
BaseExcess                 NaN
HCO3                 22.000000
FiO2                       NaN
pH                         NaN
PaCO2                      NaN
SaO2                       NaN
AST                        NaN
BUN                 100.000000
Alkalinephos               NaN
Calcium               7.900000
Chloride            113.000000
Creatinine            2.500000
Bilirubin_direct           NaN
Glucose              78.000000
Lactate                    NaN
Magnesium             2.500000
Phosphate             4.400000
Potassium             5.100000
Bilirubin_total            NaN
TroponinI                  NaN
Hct                  27.800000
Hgb                   9.700000
PTT                        NaN
WBC                  11.000000
Fibrinog

We note two demographic features, age and gender.
We note 4 features that tell us about the patients medical circumstances, which unit (unit1 and unit2), when the patient was admitted (HospAdmTime) and how long the patient stayed (ICULOS)

In [26]:
example_df.columns

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel'],
      dtype='object')

We will note that most of our data seems to be continuous with 4 categorical features (the last categorical feature is the sepsis label)

In [66]:
for data_type in example_df.dtypes.unique():
    print(str(data_type))
    print((example_df.dtypes == data_type).sum())

float64
36
int64
5


For further analysis, we will create dictionaries for the sepsis and non-sepsis population. We will run tests over these dictionaries.

In [33]:
sepsis_dict = dict()
non_sepsis_dict = dict()

In [90]:

for file in glob(train_dir + '/patient_*.psv'):
    temp_df = pd.read_csv(file, sep='|')
    # get patient ID
    patient_ID = file[ file.find("_")+1 : file.find(".") ]
    if  temp_df.SepsisLabel.sum() > 0:
        first_sepsis_row = temp_df.shape[0] - temp_df.SepsisLabel.sum()
        sepsis_dict[patient_ID] = dict()
        sepsis_dict[patient_ID]['age'] = temp_df.loc[0, 'Age']
        sepsis_dict[patient_ID]['gender'] = temp_df.loc[0, 'Gender']
        sepsis_dict[patient_ID]['unit'] = 1 if temp_df.loc[0, 'Unit1'] == 1 else 2 if temp_df.loc[0, 'Unit2'] == 1 else np.nan
        sepsis_dict[patient_ID]['HospAdmTime'] = temp_df.loc[0, 'HospAdmTime']

        sepsis_dict[patient_ID]['Sepsis ICULOS'] = temp_df.loc[first_sepsis_row, 'ICULOS']
        sepsis_dict[patient_ID]['Final ICULOS'] = temp_df.loc[temp_df.shape[0]-1, 'ICULOS']

        sepsis_dict[patient_ID]['Null Percentages'] = temp_df.iloc[:first_sepsis_row, :].count() / temp_df.iloc[:first_sepsis_row, :].shape[0]
        sepsis_dict[patient_ID]['Means'] = temp_df.iloc[:first_sepsis_row, :].mean()[:-7]
        sepsis_dict[patient_ID]['Vars'] = temp_df.iloc[:first_sepsis_row, :].var()[:-7]
    else:
        non_sepsis_dict[patient_ID] = dict()
        non_sepsis_dict[patient_ID]['age'] = temp_df.loc[0, 'Age']
        non_sepsis_dict[patient_ID]['gender'] = temp_df.loc[0, 'Gender']
        non_sepsis_dict[patient_ID]['unit'] = 1 if temp_df.loc[0, 'Unit1'] == 1 else 2 if temp_df.loc[0, 'Unit2'] == 1 else np.nan
        non_sepsis_dict[patient_ID]['HospAdmTime'] = temp_df.loc[0, 'HospAdmTime']
        
        non_sepsis_dict[patient_ID]['Final ICULOS'] = temp_df.loc[temp_df.shape[0]-1, 'ICULOS']

        non_sepsis_dict[patient_ID]['Null Percentages'] = temp_df.count() / temp_df.shape[0]
        non_sepsis_dict[patient_ID]['Means'] = temp_df.mean()[:-7]
        non_sepsis_dict[patient_ID]['Vars'] = temp_df.var()[:-7]

In [36]:
print(f'Propensity: {len(sepsis_dict) / len(non_sepsis_dict)}')
print(f'Sepsis patients: {len(sepsis_dict)}')
print(f'Patients with no record of sepsis: {len(non_sepsis_dict)}')
print(f'Total cohort: {len(sepsis_dict) + len(non_sepsis_dict)}')

Propensity: 0.07613666935700834
Sepsis patients: 1415
Patients with no record of sepsis: 18585
Total cohort: 20000


In [119]:
sepsis_df = pd.DataFrame.from_dict(data=sepsis_dict, orient='index').drop(columns=['Null Percentages', 'Means', 'Vars'])
sepsis_df['Sepsis'] = 1
non_sepsis_df = pd.DataFrame.from_dict(data=non_sepsis_dict, orient='index').drop(columns=['Null Percentages', 'Means', 'Vars'])
non_sepsis_df['Sepsis'] = 0
all_df = pd.concat([sepsis_df, non_sepsis_df])
all_df.head()

,age,gender,unit,HospAdmTime,Sepsis ICULOS,Final ICULOS,Sepsis
1,65.79,1,NaN,-0.02,26.0,35,1
10007,73.00,0,1.0,-7.08,34.0,43,1
10027,64.00,0,NaN,0.00,203.0,212,1
1003,34.00,1,2.0,-11.69,6.0,15,1
10089,49.00,1,2.0,-26.24,139.0,148,1


In [121]:
sepsis_df.head()
sepsis_summary_table = sepsis_df.mean()
sepsis_summary_table['unit1'] = (sepsis_df.unit == 1).mean()
sepsis_summary_table['unit2'] = (sepsis_df.unit == 2).mean()
non_sepsis_summary_table = non_sepsis_df.mean()
non_sepsis_summary_table['unit1'] = (non_sepsis_df.unit == 1).mean()
non_sepsis_summary_table['unit2'] = (non_sepsis_df.unit == 2).mean()
summary_table = pd.concat([sepsis_summary_table, non_sepsis_summary_table], axis=1).transpose()
summary_table

,age,gender,unit,HospAdmTime,Sepsis ICULOS,Final ICULOS,Sepsis,unit1,unit2
0,62.250940,0.580919,1.405405,-74.686587,50.504594,59.032509,1.0,0.326502,0.222615
1,61.623673,0.553565,1.513305,-49.169890,NaN,37.301157,0.0,0.302125,0.318644


In [129]:
print()
print()
print()
print()

822
10288
593
8297


We find that gender is correlated to Sepsis at a p-value of 0.049. Based on the literature, men are more succeptible to sepsis and we find that to be the case here.

In [137]:
gender_data = [[((all_df.gender == 1) & (all_df.Sepsis == 1)).sum(), ((all_df.gender == 1) & (all_df.Sepsis == 0)).sum()], 
               [((all_df.gender == 0) & (all_df.Sepsis == 1)).sum(), ((all_df.gender == 0) & (all_df.Sepsis == 0)).sum()]]
print(gender_data)
stat, p, dof, expected = chi2_contingency(gender_data)
print(p)
print(f'{((all_df.gender == 1) & (all_df.Sepsis == 1)).sum()/(all_df.gender == 1).sum()} % of men (n={(all_df.gender == 1).sum()}) with sepsis vs. {((all_df.gender == 0) & (all_df.Sepsis == 1)).sum()/(all_df.gender == 0).sum()}% women (n={(all_df.gender == 0).sum()}) with sepsis in data')


[[822, 10288], [593, 8297]]
0.04902474059626093
0.07398739873987399 % of men (n=11110) with sepsis vs. 0.06670416197975253% women (n=8890) with sepsis in data


We look at which unit the patients were treated and find that at a very significant level unit is correlated to sepsis. It is likely that unit 1 treats more ill patients. However, we will note that this information is often lacking.

In [138]:
unit_data = [[((all_df.unit == 1) & (all_df.Sepsis == 1)).sum(), ((all_df.unit == 1) & (all_df.Sepsis == 0)).sum()], 
               [((all_df.unit == 2) & (all_df.Sepsis == 1)).sum(), ((all_df.unit == 2) & (all_df.Sepsis == 0)).sum()]]
stat, p, dof, expected = chi2_contingency(unit_data)
print(p)
print(f'{((all_df.unit == 1) & (all_df.Sepsis == 1)).sum()/(all_df.unit == 1).sum()} % of unit1 (n={(all_df.unit == 1).sum()}) with sepsis vs. {((all_df.unit == 2) & (all_df.Sepsis == 1)).sum()/(all_df.unit == 2).sum()}% unit2 (n={(all_df.unit == 2).sum()}) with sepsis in data')


7.212712069706973e-09
0.07602435412209972 % of unit1 (n=6077) with sepsis vs. 0.050505050505050504% unit2 (n=6237) with sepsis in data
